<center>
    <h1>
پروژه ی چهارم
        <br>
سیستم های توصه گر      
    <h1/>
    <center/>

#### اولین کار در ابتدای هر پروژه، خواندن کتابخانه های لازم است


In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


## آماده سازی دیتاست
##### این بخش بر اساس content-based طرح ریزی شده است.


#### اول : خواندن اطلاعات از دیتاست

In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### پاک سازی دیتاست
در این دیتا ست نام فیلم و سال تولید هر دو در یک ستون قرار دارند که منجر به پایین آمدن خوانایی جدول ها می شود
با استفاده از ReGex نام فیلم و سال تولید را جدا کرده و در یک ستون جدا می نویسم
مشخصا در تعریف regex پرانتز ها هم لحاظ کردیم تا اگر در نام فیلم سال وجود دارد (مثال: 2001, space oddyssy) اشتباهی حذف نشود.
و در آخر فضاهای خالی اضافی را نیز حذف یا همان `strip` می کنیم.<br>
گزینه ی *Expand* در صورت `True` بودن به ازای هر گروه یک ستون بر می گرداند.


In [3]:
movies_df['year'] = movies_df.title.str.extract(r'(\(\d\d\d\d\))' ,expand=False)

movies_df['year'] = movies_df.year.str.extract(r'(\d\d\d\d)',expand=False)
# remove from title
movies_df['title'] = movies_df.title.str.replace(r'(\(\d\d\d\d\))', '')

movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

C:\Users\AMIRHU~1\AppData\Local\Temp/ipykernel_8636/4167376218.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace(r'(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


<h4>
ژانر ها تماما در یک ستون نوشته شده است که این پردازش را سخت می کند.<br><br>
در این مرحله ستون Genres را برای جدا سازی آماده می کنیم. <br><br>
ابتدا تمام ژانر های در یک لیست قرار می می دهیم. <br><br>
    در جدول ژانر ها به '|' جدا شده اند
</h4>

In [4]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


حتی این شکل از ستون ژانر نیز برای یادگیری مناسب نیست 
با استفاده از تکنیک زیر برای هر ژانر یک ستون تعریف می کنیم، و اگر فیلمی شامل این ژانر می شد مقدار *1* و اگر زیرشاخه ی این ژانر نبود مقدار *0* را قرار می دهیم.

In [5]:
moviesWithGenres_df = movies_df.copy()

# For every row in the dataframe, 
# iterate through the list of genres 
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df = moviesWithGenres_df.drop('genres', axis=1)
moviesWithGenres_df.head()

,movieId,title,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# ورود جدول تست سیستم
حالا برای ساخت یک جدول تست (کاربر تست) اقدام می کنیم.
ابتدا جدولی با نام `userInput` و بعد آن را به یک جدول pandas با نام `inputMovies` تبدیل می کنیم.

In [7]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'V for Vendetta', 'rating':4.5},
            {'title':'Citizen Kane', 'rating':5},
            {'title':"Sabrina", 'rating':5},
            {'title':'Cat on a Hot Tin Roof', 'rating':4.5},
            {'title':'Last of the Mohicans, The', 'rating':4 }
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,V for Vendetta,4.5
2,Citizen Kane,5.0
3,Sabrina,5.0
4,Cat on a Hot Tin Roof,4.5
5,"Last of the Mohicans, The",4.0


## اضافه کردن id فیلم ها به جدول تست
حالا باید ببینم که فیلم های کاربر در دیتا ست ما وجود دارند یا خیر و درصورت موجود بود ID مربوط به فیلم را در دیتاست را برگرداند. <br>
برای اینکار ابتدا سطر هایی که حاوی نام فیلم می باشد را فیلتر می کنیم و بعد از پیدا کردن آن رابا دیتاست ست درهم یا `merge` می کنیم. <br>
و در آخر ستون های غیرضروری را حذف می کنیم.<br>
*توجه*<br>
اگر فیلم که در دیتاست user بود ولی در جدول زیر نبود به یکی از دو دلیل است:
* املا ی آن اشتباه است
* اصلا در دیتاست اصلی وجود ندارد

In [8]:
#Filtering
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#merging
inputMovies = pd.merge(inputId, inputMovies)
#Dropping
inputMovies = inputMovies.drop('genres', axis=1).drop('year', axis=1)
inputMovies

,movieId,title,rating
0,7,Sabrina,5.0
1,915,Sabrina,5.0
2,923,Citizen Kane,5.0
3,971,Cat on a Hot Tin Roof,4.5
4,1408,"Last of the Mohicans, The",4.0
5,1968,"Breakfast Club, The",5.0
6,44191,V for Vendetta,4.5


# شروع یادگیری
اکنون فیلم هایی که کاربر تماشا کرده است از Data Frame حاوی ژانر ها با مقادیر باینری نیز هست میگیریم.

In [9]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
6,7,Sabrina,1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,915,Sabrina,1954,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705,923,Citizen Kane,1941,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
744,971,Cat on a Hot Tin Roof,1958,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1084,1408,"Last of the Mohicans, The",1992,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1445,1968,"Breakfast Club, The",1985,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6151,44191,V for Vendetta,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


ما تنها نیاز به جدول ژانر ها داریم، برای این کار نیاز داریم که داده ها را تا حدی پاک سازی کنیم.<br>
داده های  که انجام شدند:
* اندیس گذاری مجدد
* حذف ستون ID فیلم ها
* حذف ستون عنوان فیلم 
* حذف ستون ژانر
* حذف ستون سال تولید

In [10]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', axis=1).drop('title', axis=1).drop('year', axis=1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# حالا برای شروع یادگیری آماده ایم

برای این کار هر ژانر را تبدیل به یک وزن می کنیم.
<br>
برای این کار "نظر های کاربر ورودی و ضرب آنها در جدول ژانر ورودی و سپس جمع جدول با ستون " انجام می دهیم.
<br>
این عملیات در واقع ضرب نقطه ای ماتریس `dot` نامیده می شود.
ضرب نقطه ای ضرب یک ماتریس در یک بردار است.

In [11]:
inputMovies['rating']

0    5.0
1    5.0
2    5.0
3    4.5
4    4.0
5    5.0
6    4.5
Name: rating, dtype: float64

In [12]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
# Profiling User
userProfile

Adventure              0.0
Animation              0.0
Children               0.0
Comedy                15.0
Fantasy                0.0
Romance               14.0
Drama                 14.5
Action                 8.5
Crime                  0.0
Thriller               4.5
Horror                 0.0
Mystery                5.0
Sci-Fi                 4.5
War                    4.0
Musical                0.0
Documentary            0.0
IMAX                   4.5
Western                4.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

حالا برای تمام "ترجیحات" کاربر یک وزن داریم
<br>
به این پروفیال سازی برای کاربر گویند.
<br>
از این جدول برای پیشنهاد فیلم به کاربر می توان استفاده کرد.

حالا باید از جدول اصلی، جدول ژانر را استخارج می کنیم.

In [13]:

genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
# Drop the unnecessary 
genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('year', axis=1)

print(genreTable.shape)
genreTable.head()

(9742, 20)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


اکنون لیست های فیلم و ژانر، برای اوزان یک میانگین بر اساس پروفایل کاربر ورودی میگیریم و 20 فیلم پیشنهادی به نمایش می گذاریم.

In [14]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.191083
2    0.000000
3    0.369427
4    0.554140
5    0.191083
dtype: float64

In [15]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
4956     0.719745
4719     0.719745
26236    0.713376
72142    0.662420
42015    0.662420
dtype: float64

### اکنون جدول پیشنهادی :

In [16]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
505,587,Ghost,"[Comedy, Drama, Fantasy, Romance, Thriller]",1990
1134,1484,"Daytrippers, The","[Comedy, Drama, Mystery, Romance]",1996
1394,1912,Out of Sight,"[Comedy, Crime, Drama, Romance, Thriller]",1998
2903,3893,Nurse Betty,"[Comedy, Crime, Drama, Romance, Thriller]",2000
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
3608,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
4009,5666,"Rules of Attraction, The","[Comedy, Drama, Romance, Thriller]",2002
4655,6954,"Barbarian Invasions, The (Les invasions barbares)","[Comedy, Crime, Drama, Mystery, Romance]",2003
5029,7831,Another Thin Man,"[Comedy, Crime, Drama, Mystery, Romance]",1939
5033,7835,Song of the Thin Man,"[Comedy, Crime, Drama, Musical, Mystery, Romance]",1947
